# Yelp API Core

Susan Shin

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi

In [3]:
with open('/Users/susanshin/.secret/yelp_api.json','r') as f:
    login = json.load(f)

In [4]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [7]:
yelp = YelpAPI(login['api-key'],timeout_s= 5.0)

In [8]:
location = 'San Francisco, CA 94133'
term = 'italian'

In [10]:
FOLDER ='Data/'
os.makedirs(FOLDER,exist_ok=True)

In [11]:
location.split(',')[0]

'San Francisco'

In [13]:
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [14]:
JSON_FILE

'Data/San Francisco-italian.json'

In [16]:
file_exists = os.path.isfile(JSON_FILE)

if file_exists == False:
    folder = os.path.dirname(JSON_FILE)
    if len(folder)>0:
        os.makedirs(folder, exist_ok=True)
        print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
        with open (JSON_FILE, 'w') as f:
            json.dump([],f)
else:
    print(f"[i]{JSON_FILE} already exists")

[i]Data/San Francisco-italian.json already exists


In [17]:
results = yelp.search_query(term = term,location = location)

In [18]:
type(results)

dict

In [19]:
len(results)

3

In [20]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [21]:
results["total"]

463

In [22]:
results_per_page = len(results['businesses'])
results_per_page

20

In [23]:
import time, math

n_pages = math.ceil((results['total'])/ results_per_page)
n_pages

24

In [24]:
for i in tqdm_notebook( range(1,n_pages+1)):
    try:
        time.sleep(.2)
        with open (JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        n_results = len(previous_results)
        results = yelp.search_query(location = location, term = term, offest = n_results+1)
        previous_results.extend(results['businesses'])
        with open (JSON_FILE, 'w') as f:
            json.dump(previous_results,f)
    except Exception as e:
        print('[!] ERROR:', e)
        

  0%|          | 0/24 [00:00<?, ?it/s]

In [25]:
df = pd.read_json(JSON_FILE)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8dUaybEPHsZMgr1iKgqgMQ,sotto-mare-san-francisco-4,Sotto Mare,https://s3-media3.fl.yelpcdn.com/bphoto/o3hIcG...,False,https://www.yelp.com/biz/sotto-mare-san-franci...,4695,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 37.79979, 'longitude': -122.40834}","[delivery, pickup]",$$,"{'address1': '552 Green St', 'address2': '', '...",14153983181,(415) 398-3181,421.499100
1,K9XVDlPNhrrSVEJN7uWqJQ,the-italian-homemade-company-san-francisco-8,The Italian Homemade Company,https://s3-media2.fl.yelpcdn.com/bphoto/_5-c4n...,False,https://www.yelp.com/biz/the-italian-homemade-...,1818,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 37.801752, 'longitude': -122.411851}","[delivery, pickup]",$$,"{'address1': '716 Columbus Ave', 'address2': '...",14157128874,(415) 712-8874,218.158166
2,o43B4DnnQbvkdDK6AVafQg,piccolo-forno-san-francisco,Piccolo Forno,https://s3-media2.fl.yelpcdn.com/bphoto/f5cIsI...,False,https://www.yelp.com/biz/piccolo-forno-san-fra...,833,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 37.8013699308642, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '725 Columbus Ave', 'address2': '...",14157570087,(415) 757-0087,258.085989
3,6e7gMgVJRQ5rtkIbTF6TLQ,trattoria-contadina-san-francisco,Trattoria Contadina,https://s3-media4.fl.yelpcdn.com/bphoto/9mKRj1...,False,https://www.yelp.com/biz/trattoria-contadina-s...,1441,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 37.80018, 'longitude': -122.41228}",[delivery],$$,"{'address1': '1800 Mason St', 'address2': '', ...",14159825728,(415) 982-5728,383.428067
4,I8T8aoAsFQYwQiRvLBayFA,il-casaro-pizzeria-and-mozzarella-bar-north-be...,Il Casaro Pizzeria & Mozzarella Bar - North Beach,https://s3-media4.fl.yelpcdn.com/bphoto/ef3tF8...,False,https://www.yelp.com/biz/il-casaro-pizzeria-an...,1592,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 37.79852, 'longitude': -122.40739}","[delivery, pickup]",$$,"{'address1': '348 Columbus Ave', 'address2': '...",14156779455,(415) 677-9455,580.994463


In [27]:
csv_file = JSON_FILE.replace('.json', '.csv.gz')
csv_file

'Data/San Francisco-italian.csv.gz'

In [28]:
df.to_csv(csv_file, compression = 'gzip', index = False)